In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

#from sklearn.preprocessing import category_encoders as ce

df=pd.read_csv('HW4_creditcard.csv')
y =df['Class'].values
x = df.iloc[:,:-1]

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state = 42)

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

modelt = DecisionTreeClassifier()
modelt.fit(train_x,train_y)
predict_y = modelt.predict(test_x)
print("\t~~~~~Decision Tree~~~~~")
print("Accuracy score:  \t" ,(accuracy_score(test_y, predict_y)*100)," %" ,sep='')
print("Precission score:\t" ,(precision_score(test_y, predict_y)*100)," %" ,sep='')
print("Recall Score:   \t" ,(recall_score(test_y, predict_y)*100)," %" ,sep='')
print("F1 score:       \t" ,(f1_score(test_y, predict_y)*100)," %" ,sep='')

	~~~~~Decision Tree~~~~~
Accuracy score:  	99.42383037566259 %
Precission score:	84.31372549019608 %
Recall Score:   	83.22580645161291 %
F1 score:       	83.76623376623378 %


In [3]:
import tensorflow.keras as tk
from tensorflow.keras import layers
from keras.layers import Input,Dense,Dropout,Activation
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, recall_score, precision_score

train_x_np = train_x.to_numpy()
test_x_np = test_x.to_numpy()
train_y_np = train_y
test_y_np = test_y



model = Sequential()
model.add(Dense(5000,activation='relu',input_shape=(None,30)))
model.add(Dropout(0.2))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))




In [4]:
from colorama import Fore,Style
model.compile(optimizer='Adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_x_np,train_y_np)
loss = model.evaluate(test_x_np,test_y_np)[0]
acc = model.evaluate(test_x_np,test_y_np)[1]
print(Fore.BLUE+"loss function : ", loss)
print(Fore.BLUE+"accuracy : ", acc)

272/272 [==============================] - 0s 1ms/step - loss: 0.2843 - accuracy: 0.9821
loss function :  0.284307062625885
accuracy :  0.9821387529373169


In [5]:
print("Class為1的資料比數: ",df[df['Class']==1].count()[0])
print("Class為0的資料比數: ",df[df['Class']==0].count()[0])
print("1:0比率為: ",df[df['Class']==1].count()[0]/df[df['Class']==0].count()[0])

Class為1的資料比數:  492
Class為0的資料比數:  28432
1:0比率為:  0.01730444569499156


In [6]:
modelt = DecisionTreeClassifier(class_weight={0:1,1:10})
modelt.fit(train_x,train_y)
predict_y = modelt.predict(test_x)
print("~~~~~Decision Tree(modified)~~~~~")
print("Accuracy score:  \t" ,(accuracy_score(test_y, predict_y)*100)," %" ,sep='')
print("Precission score:\t" ,(precision_score(test_y, predict_y)*100)," %" ,sep='')
print("Recall Score:   \t" ,(recall_score(test_y, predict_y)*100)," %" ,sep='')
print("F1 score:       \t" ,(f1_score(test_y, predict_y)*100)," %" ,sep='')

~~~~~Decision Tree(modified)~~~~~
Accuracy score:  	99.41230698317585 %
Precission score:	83.76623376623377 %
Recall Score:   	83.22580645161291 %
F1 score:       	83.49514563106796 %


我們發現target在訓練集中的**<font color='red'>類別分布不太平衡<font>**，可能對預測準確度有影響。<br>
因此在新的decision tree中調整class_weight參數，經過多次測試，得到**<font color='red'>1:10的類別權重<font>**效果最佳

In [10]:
model.fit(train_x_np,train_y_np,class_weight={0:1,1:10})
loss = model.evaluate(test_x_np,test_y_np)[0]
acc = model.evaluate(test_x_np,test_y_np)[1]

print(Fore.BLUE+"loss function : ", loss)
print(Fore.BLUE+"accuracy : ", acc)


272/272 [==============================] - 0s 1ms/step - loss: 0.1177 - accuracy: 0.9821
loss function :  0.11771182715892792
accuracy :  0.9821387529373169


並且對於神經網路模型也進行類別權重的調整，但是經過多次調整，發現類別權重的改變似乎**<font color='red'>僅能減少loss function<font>**，卻難以提高準確度。

In [11]:
from imblearn.over_sampling import SMOTE
res_x,res_y = SMOTE(random_state=42).fit_resample(train_x,train_y)
modelt = DecisionTreeClassifier()
modelt.fit(res_x,res_y)
predict_y = modelt.predict(test_x)
print("~~~~~Decision Tree(imcremented)~~~~~")
print("Accuracy score:  \t" ,(accuracy_score(test_y, predict_y)*100)," %" ,sep='')
print("Precission score:\t" ,(precision_score(test_y, predict_y)*100)," %" ,sep='')
print("Recall Score:   \t" ,(recall_score(test_y, predict_y)*100)," %" ,sep='')
print("F1 score:       \t" ,(f1_score(test_y, predict_y)*100)," %" ,sep='')

	~~~~~Decision Tree~~~~~
Accuracy score:  	98.8015671813782 %
Precission score:	61.97183098591549 %
Recall Score:   	85.16129032258064 %
F1 score:       	71.7391304347826 %


In [12]:
res_x_np = res_x.to_numpy()
res_y_np = res_y
model.fit(res_x_np,res_y_np)
loss = model.evaluate(test_x_np,test_y_np)[0]
acc = model.evaluate(test_x_np,test_y_np)[1]

print(Fore.BLUE+"loss function : ", loss)
print(Fore.BLUE+"accuracy : ", acc)


272/272 [==============================] - 0s 1ms/step - loss: 0.4196 - accuracy: 0.9821
loss function :  0.4196315109729767
accuracy :  0.9821387529373169


除了調整類別權重，我們也嘗試使用SMOTE所提供的數據合併功能。<br>
複製將小部分相似的數據，達到數遽增量的效果，來減少類別權重的不平衡。<br>
但是**<font color='red'>測試結果卻很不顯著，過度的增量數據，反而降低預測的準確度<font>**。